## Ensemble methods（集成方法）

### 集成方法的目标是结合使用给定学习算法构建的几个基本估计器的预测，以便通过单个估计器来提高泛化/鲁棒性。
通常区分两个系统的集合方法：
* 在平均方法中，驱动原则是独立构建几个估计量，然后平均其预测。平均来说，组合估计器通常比单个基准估计器更好，因为它的方差减小。
示例： Bagging方法 ，随机树木森林 ...
* 相比之下，在提升方法中，依次构建基本估计量，并尝试减少组合估计量的偏差。动机是组合几个弱模型来产生一个强大的合奏。
示例：AdaBoost，渐变树提升，..

## Bagging元估计
在集成的算法，bagging方法形成的一类，其建立在原有训练集的随机子集黑匣子估计的几个实例，然后汇总它们各自的预测，以形成最终的预测算法。这些方法被用作减少基本估计量（例如决策树）的方差的方式，通过将随机化引入到其构建过程中，然后将其合并出来。在许多情况下，装袋方法构成了一种非常简单的改进单一模型的方式，而不需要适应基础的基本算法。由于它们提供了一种减少过度装配的方式，所以采用强大而复杂的模型（例如，完全开发的决策树），装袋方法最有效，与通常用弱模型最好的增强方法（例如，<br>
装袋方法有许多风格，但是通过绘制训练集的随机子集的方式彼此大不相同：<br>
* 当数据集的随机子集被绘制为样本的随机子集时，则该算法被称为粘贴[B1999]。
* 当抽取样品时，该方法称为Bagging [B1996]。
* 当数据集的随机子集被绘制为特征的随机子集时，则该方法被称为随机子空间[H1998]。
* 最后，当基本估计器建立在样本和特征的子集上时，该方法被称为随机补丁[LG2012]。
在scikit学习，套袋方法是提供一个统一的 BaggingClassifier元估计（相应的BaggingRegressor），作为输入与指定绘制随机子集的策略参数沿用户指定的基本估计。特别地，max_samples 并且max_features控制子集的大小（在样本和特征方面），同时bootstrap并bootstrap_features控制是否抽出样本和特征。当使用可用样本的子集时，可以通过设置通过样品外样品估算泛化精度oob_score=True。例如，下面的代码片段展示了如何实例化一个KNeighborsClassifier基准估计器的装袋集合 ，每一个基于估计的50％的随机子集和50％的特征。

In [1]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging=BaggingClassifier(KNeighborsClassifier(),max_samples=0.5,max_features=0.5)

## 随机树的森林
该sklearn.ensemble模块包括基于随机决策树的两个平均算法：RandomForest算法和Extra-Trees方法。这两种算法都是针对树木设计的扰动和组合技术[B1998]。这意味着通过在分类器构造中引入随机性来创建一组不同的分类器。集合的预测作为单个分类器的平均预测给出。
作为其他分类器，森林分类器必须配备两个数组：保存训练样本的大小的稀疏或密集数组X以及保存训练样本的目标值（类标签）的大小的数组Y ：[n_samples, n_features][n_samples]

In [3]:
from sklearn.ensemble import  RandomForestClassifier
help(RandomForestClassifier)

Help on class RandomForestClassifier in module sklearn.ensemble.forest:

class RandomForestClassifier(ForestClassifier)
 |  A random forest classifier.
 |  
 |  A random forest is a meta estimator that fits a number of decision tree
 |  classifiers on various sub-samples of the dataset and use averaging to
 |  improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is always the same as the original
 |  input sample size but the samples are drawn with replacement if
 |  `bootstrap=True` (default).
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------
 |  n_estimators : integer, optional (default=10)
 |      The number of trees in the forest.
 |  
 |  criterion : string, optional (default="gini")
 |      The function to measure the quality of a split. Supported criteria are
 |      "gini" for the Gini impurity and "entropy" for the information gain.
 |      Note: this parameter is tree-specific.
 |  
 |  max_features : int, fl

In [4]:
X = [[0, 0], [1, 1]]
Y = [0, 1]
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X, Y) 

In [5]:
clf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## 随机森林 
在随机森林（参见RandomForestClassifier和 RandomForestRegressor类）中，集合中的每个树都是从训练集中替换（即，引导样本）抽取的样本构建的。另外，当在树的构建期间分割节点时，所选择的分割不再是所有特征中最好的分割。相反，所选的分割是特征的随机子集中的最佳分割。由于这种随机性，森林的偏差通常略微增加（相对于单个非随机树的偏差），但是由于平均，其方差也减小，通常大于补偿偏差的增加，从而产生更好的模型。
与原始出版物[B2001]相反，scikit学习实现通过对分类器进行平均，将其概率预测相结合，而不是让每个分类器对单个类进行投票。

## 极端随机树 
在非常随机的树（参见ExtraTreesClassifier 和ExtraTreesRegressor类）中，随机性进一步分离计算的方式。如在随机森林中，使用候选特征的随机子集，而不是寻找最具有歧视性的阈值，而是为每个候选特征随机绘制阈值，并且将这些随机生成的阈值中的最佳值作为分割规则。这通常允许更多地减少模型的方差，牺牲稍微更大的偏差增加：

In [6]:
>>> from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

X, y = make_blobs(n_samples=10000, n_features=10, centers=100,
    random_state=0)

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2,
    random_state=0)
scores = cross_val_score(clf, X, y)
print('DecisionTreeClassifier:'+str(scores.mean()))                            

clf = RandomForestClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y)
print('RandomForestClassifier:'+str(scores.mean()))                             

clf = ExtraTreesClassifier(n_estimators=10, max_depth=None,
    min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y)
print('ExtraTreesClassifier:'+str(scores.mean()))  

DecisionTreeClassifier:0.979408793821
RandomForestClassifier:0.999607843137
ExtraTreesClassifier:0.999898989899


### 参数
使用这些方法时要调整的主要参数是n_estimators 和max_features。前者是森林里的树木数量。越大越好，而且计算时间越长。此外，请注意，超过关键数量的树木，结果将停止显着改善。后者是分割节点时要考虑的特征的随机子集的大小。偏差减小越大，偏差越大。经验良好的默认值max_features=n_features 用于回归问题，max_features=sqrt(n_features)分类任务（n_features数据中的功能数量在哪里）。通常max_depth=None 结合min_samples_split=1（即，完全开发树木）时，会获得良好的效果。请记住，这些值通常不是最佳的，并且可能导致使用大量RAM的模型。应始终交叉验证最佳参数值。bootstrap=True另外，请注意，在随机林中，默认使用引导样本（ ），而外部树的默认策略是使用整个数据集（bootstrap=False）。当使用引导抽样时，可以在左边或外面的样本上估算泛化精度。这可以通过设置启用oob_score=True。当使用引导抽样时，可以在左边或外面的样本上估算泛化精度。这可以通过设置启用。当使用引导抽样时，可以在左边或外面的样本上估算泛化精度。这可以通过设置启用。
### 并行化
最后，该模块还具有树的并行构建和通过n_jobs 参数的预测的并行计算。如果n_jobs=k然后计算被划分为 k作业，并k在机器的内核上运行。如果n_jobs=-1 然后使用机器上可用的所有核心。请注意，由于进程间通信开销，加速可能不是线性的（即，k不幸的是使用作业不会k那么快）。当建立大量的树，或者构建单个树需要相当长的时间（例如大型数据集）时，仍然可以实现显着的加速。

## 全部随机树嵌入 
RandomTreesEmbedding实现数据的无监督转换。使用完全随机树的森林，RandomTreesEmbedding 通过数据点最后的叶子的索引来对数据进行编码。然后，该索引以K方式编码，导致高维稀疏二进制编码。可以非常有效地计算该编码，然后可以将其用作其他学习任务的基础。可以通过选择树的数量和每棵树的最大深度来影响代码的大小和稀疏性。对于集合中的每个树，编码包含一个条目。编码的大小最多是森林中最大叶数。n_estimators * 2 ** max_depth<br>
随着相邻数据点更可能位于树的同一叶内，变换执行隐式非参数密度估计。<br>
例子：
* 使用全部随机树进行哈希特征变换
* 手写数字上的歧管学习：本地线性嵌入，Isomap ...比较手写数字上的非线性降维技术。
* 与树的集合的特征变换比较了监督和无监督的基于树的特征变换。


## AdaBoost
该模块sklearn.ensemble包括1995年由Freund和Schapire [FS1995]推出的流行的升压算法AdaBoost 。
AdaBoost的核心原则是在数据的反复修改版本上适应一系列弱学习者（即，仅比稍微优于随机猜测的模型，如小决策树）。然后将所有这些预测通过加权多数票（或总和）合并以产生最终预测。在每一个所谓的提高迭代的数据修改由施加权重的W_1，W_2......，w_N 每个训练样本。最初，这些权重都被设置为 ，所以第一步只是训练一个弱学习者的原始数据。对于每个连续迭代，样本权重被单独修改，并且将学习算法重新应用于重新加权的数据。在给定的步骤， 那些在前一步骤诱发的增强模型不正确预测的训练样本的权重增加，而正确预测的那些训练样本的权重则会降低。随着迭代的进行，难以预测的示例将会受到越来越大的影响。因此，每个后续的弱小的学习者被迫集中在序列[HTF]中被先前错过的例子。

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier

iris = load_iris()
clf = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(clf, iris.data, iris.target)
scores.mean()    

0.95996732026143794

## 渐变树提升
渐变树提升 或渐变增强回归树（GBRT）是推广到任意可微分损失函数的推广。GBRT是一个准确有效的现成程序，可用于回归和分类问题。梯形树提升模型用于各种领域，包括网络搜索排名和生态学。<br>
GBRT的优点是：
* 混合型数据的自然处理（=异构特征）
* 预测力
* 输出空间异常值的鲁棒性（通过强大的损失函数）<br>
GBRT的缺点是：
* 由于升压的顺序性，可扩展性几乎不能并行化。
* 该模块sklearn.ensemble提供了通过梯度增强回归树进行分类和归一化的方法。

In [8]:
#分类
#GradientBoostingClassifier支持二进制和多类分类。以下示例显示如何将具有100个决策树枝的渐变增强分类器作为弱学习者：
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
 
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)   


0.91300000000000003

In [9]:
# 回归
# GradientBoostingRegressor支持一些 可以通过参数指定的 不同的回归 损失函数loss ; 回归的默认损失函数为最小二乘（'ls'）。 
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor

X, y = make_friedman1(n_samples=1200, random_state=0, noise=1.0)
X_train, X_test = X[:200], X[200:]
y_train, y_test = y[:200], y[200:]
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
    max_depth=1, random_state=0, loss='ls').fit(X_train, y_train)
mean_squared_error(y_test, est.predict(X_test))   

5.0091548599603213

## 投票分类器
投票分类器实施背后的想法是将概念上不同的机器学习分类器结合起来，并使用多数投票或平均预测概率（软投票）来预测类标签。这样的分类器可以用于一组性能良好的模型，以平衡其个人的弱点。
### 大多数类标签（大多数/硬投票）
在多数投票中，特定样本的预测类标签是表示每个分类器预测的类标签的大多数（模式）的类标签。
例如，如果给定样本的预测是
* 分类器1 - >类1
* 分类器2 - >类1
* 分类器3 - >类2
VotingClassifier（with voting='hard'）将根据大多数类标签将样本分类为“类1”。
在系统的情况下，VotingClassifier将根据升序排序顺序选择该类。例如，在以下情况下
* 分类器1 - >类2
* 分类器2 - >类1
类标签1将被分配给样品。
用法
以下示例显示如何适合大多数规则分类器：

In [11]:
from sklearn import datasets
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

iris = datasets.load_iris()
X, y = iris.data[:, 1:3], iris.target

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.90 (+/- 0.05) [Logistic Regression]
Accuracy: 0.93 (+/- 0.05) [Random Forest]
Accuracy: 0.91 (+/- 0.04) [naive Bayes]
Accuracy: 0.95 (+/- 0.05) [Ensemble]


In [12]:

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from itertools import product
from sklearn.ensemble import VotingClassifier

# Loading some example data
iris = datasets.load_iris()
X = iris.data[:, [0,2]]
y = iris.target

# Training classifiers
clf1 = DecisionTreeClassifier(max_depth=4)
clf2 = KNeighborsClassifier(n_neighbors=7)
clf3 = SVC(kernel='rbf', probability=True)
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2), ('svc', clf3)], voting='soft', weights=[2,1,2])

clf1 = clf1.fit(X,y)
clf2 = clf2.fit(X,y)
clf3 = clf3.fit(X,y)
eclf = eclf.fit(X,y) 

## 使用VotingClassifier与GridSearch 
所述VotingClassifier也可以加合使用GridSearch为了调整个体估计量的超参数：

In [13]:
from sklearn.model_selection import GridSearchCV
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')

params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200],}

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(iris.data, iris.target) 